In [1]:
# SELECT DISTINCT ?personVal ?famVal ?orgVal ?orgOVal WHERE {
#     GRAPH <http://tbio.orient.cas.cz/chrj_pr> {
#         ?person tbio:isMemberOfFamily ?p .
#         ?p tbio:hasFamily ?fam .
#         FILTER(!sameterm(?fam, tbio:WufengLinFamily)) .
#     }
#     OPTIONAL {
#         ?person ?orgP ?orgEvt .
#         FILTER(sameterm(?orgP, tbio:attendsSchool) || sameterm(?orgP, tbio:hasEmployment) || sameterm(?orgP, tbio:hasMembership))
#         ?orgEvt ?orgAt ?org .
#         FILTER(sameterm(?orgAt, tbio:educationAt) || sameterm(?orgAt, tbio:employmentAt) || sameterm(?orgAt, tbio:hasMembershipAt))
#         ?org rdf:type ?orgO .
#     }
#     BIND(STR(?person) AS ?personStr) .
#     BIND(REPLACE(?personStr, "http://tbio.orient.cas.cz#", "") AS ?personVal) .
#     BIND(STR(?fam) AS ?famStr) .
#     BIND(REPLACE(?famStr, "http://tbio.orient.cas.cz#", "") AS ?famVal) .
#     BIND(STR(?org) AS ?orgStr) .
#     BIND(REPLACE(?orgStr, "http://tbio.orient.cas.cz#", "") AS ?orgVal) .
#     BIND(STR(?orgO) AS ?orgOStr) .
#     BIND(REPLACE(?orgOStr, "http://tbio.orient.cas.cz#", "") AS ?orgOVal) .
# }

In [2]:
import pandas as pd

In [3]:
filepath = '/Volumes/backup_128G/z_repository/TBIO_data/RequestsFromTana/20190521_perFamOrgSocPer'

filename = 'personFamilyOrgPeriod-20190516.tsv'

read_filename = '{0}/{1}'.format(filepath, filename)

In [4]:
workDf = pd.read_csv(read_filename, delimiter='\t')
workDf = workDf.dropna().reset_index(drop=True)
workDf.shape, workDf.head()

((8461, 4),   ?personVal ?famVal     ?orgVal                      ?orgOVal
 0        林熊祥    嚴復家系     林本源製糖會社  JapaneseColonialOrganization
 1        林熊祥    嚴復家系     林本源製糖會社               PrivateBusiness
 2        林熊祥    嚴復家系  台灣大有物產株式會社  JapaneseColonialOrganization
 3        林熊祥    嚴復家系  台灣大有物產株式會社               PrivateBusiness
 4        林熊祥    嚴復家系    台北商事株式會社  JapaneseColonialOrganization)

In [5]:
# SELECT DISTINCT ?s WHERE {
#     GRAPH ?graph {
#         ?s rdfs:subClassOf tbio:Division_Period .
#     }
# }

Division_Period = [
    'JapaneseColonialOrganization',
    'EarlyPostWarOrganization',
    'RepublicanOrganization',
    'BothPeriods',
    'PreRetrossecionRepublicanOrganization',
    'QingPeriodOrganization',
    'UnknownPeriodOrganization'
]

In [6]:
# SELECT DISTINCT ?s WHERE {
#     GRAPH ?graph {
#         ?s rdfs:subClassOf tbio:SocietalSector .
#     }
# }

SocietalSector = [
    'Police',
    'ArmedForces',
    'Hospital',
    'ReligiousOrganization',
    'MassMedia',
    'PublicAdministration',
    'PrivateBusiness',
    'VoluntaryAssociation',
    'Academia_Education',
    'StateBusiness',
    'CulturalOrganization',
    'Politics',
    'UnknownSector'    
]

In [7]:
workList = []
for idx in range(0, len(workDf)):
    row = workDf.loc[idx]

    person = str(row['?personVal'])
    family = str(row['?famVal'])
    org = str(row['?orgVal'])
    
    if org == '':
        continue
    orgO = str(row['?orgOVal'])

    workList.append([person, family, org, orgO])
    
print(len(workList))

8461


In [8]:
workList[0:5]

[['林熊祥', '嚴復家系', '林本源製糖會社', 'JapaneseColonialOrganization'],
 ['林熊祥', '嚴復家系', '林本源製糖會社', 'PrivateBusiness'],
 ['林熊祥', '嚴復家系', '台灣大有物產株式會社', 'JapaneseColonialOrganization'],
 ['林熊祥', '嚴復家系', '台灣大有物產株式會社', 'PrivateBusiness'],
 ['林熊祥', '嚴復家系', '台北商事株式會社', 'JapaneseColonialOrganization']]

# 全臺詩社

In [9]:
poemOrgs = '全臺詩詩社表格-20180717.xlsx'

read_poemOrgs = '{0}/{1}'.format(filepath, poemOrgs)

In [10]:
poemDf = pd.read_excel(read_poemOrgs)
poemDf.shape, poemDf.head()

((340, 15),
    詩社名稱 別名                      創始人               創社時間       結束時間  \
 0    瀛社  無                 謝汝銓\n林馨蘭  1909.3.7-1909.3.7         迄今   
 1    櫟社  無                 林朝崧\n賴紹堯               1902          ?   
 2   崇正社  無                      許南英          1878-1878  1890-1890   
 3  浪吟詩社  無  許南英\n蔡國琳\n胡殿鵬\n趙鍾麒\n謝維巖          1891-1891  1906-1906   
 4    南社  無             連橫\n趙鍾麒\n謝維巖               1906          ?   
 
                  創社地點                               歷任社長  \
 0    平樂遊（臺灣臺北市萬華區西昌街）  洪以南\n謝汝銓\n魏清德\n李建興\n杜萬吉\n黃鷗波\n林正三   
 1        霧峰（臺灣臺中市霧峰區）                 蔡見先\n賴紹堯\n傅錫祺\n林獻堂   
 2  竹溪寺（臺灣臺南市南區體育路87號）                                  無   
 3                  不詳                                  無   
 4          臺南市（臺灣臺南市）                                蔡國琳   
 
                                                   社員  \
 0  洪以南\n林馨蘭\n陳宗賦\n魏清德\n謝汝銓\n謝尊五\n顏笏山\n黃贊鈞\n許梓桑\n黃...   
 1  林朝崧\n賴紹堯\n林資修\n蔡見先\n呂敦禮\n林資銓\n陳懷澄\n陳錫金\n陳瑚\n陳貫...   
 2  丘逢甲\n許南英\n蕭逢源\n陳望曾\n陳日翔

In [11]:
poemDic = {}
for idx in range(0, len(poemDf)):
    row = poemDf.loc[idx]

    poem = str(row['詩社名稱'])
    members = str(row['創始人']).split('\n')
    members.extend(str(row['歷任社長']).split('\n'))
    members.extend(str(row['社員']).split('\n'))
    
    if poem not in poemDic:
        poemDic[poem] = members
    else:
        print("DUP:", poem)

# poemDic

DUP: 文山吟社
DUP: 樹村吟社
DUP: 白沙吟社
DUP: 香草吟社
DUP: 香草吟社
DUP: 梅社
DUP: 鶯社
DUP: 西螺吟社
DUP: 岡山吟社
DUP: 南雅社
DUP: 竹林吟社
DUP: 竹風吟社
DUP: 青蓮吟社
DUP: 柏社
DUP: 梅社


In [12]:
for poem in poemDic:
    members = poemDic[poem]
    
    for person in members:
        if person == '不詳':
            continue

        row = [person, "poem", poem, 'CulturalOrganization']
        if row not in workList:
            workList.append(row)
#         else:
#             print("DUP:", row)
            
        row = [person, "poem", poem, 'UnknownPeriodOrganization']
        if row not in workList:
            workList.append(row)
#         else:
#             print("DUP:", row)

# Read person-family map table

In [13]:
familymembers = 'Familymembers_20190521.xlsx'

read_familymembers = '{0}/{1}'.format(filepath, familymembers)

In [14]:
fmDf = pd.read_excel(read_familymembers)
fmDf.shape, fmDf.head()

((4546, 2),   personStr familyStr
 0       何佩娟      何傳家系
 1       何佩珊      何傳家系
 2       何佩珍      何傳家系
 3        何傳      何傳家系
 4       何培庭      何傳家系)

In [15]:
fmDic = {}
for idx in range(0, len(fmDf)):
    row = fmDf.loc[idx]

    person = str(row['personStr'])
    family = str(row['familyStr'])
    
    if person not in fmDic:
        fmDic[person] = family
    elif family != fmDic[person]:
        print("Dup:", person, family, fmDic[person])

# fmDic

In [16]:
def getPeriod(INperson, INorg):
    for row in workList:
        person = row[0]
#         family = row[1]
        org = row[2]
        orgO = row[3]

        if person == INperson and org == INorg:
            if orgO in Division_Period:
                return orgO
    return ''

def getSocietal(INperson, INorg):
    for row in workList:
        person = row[0]
#         family = row[1]
        org = row[2]
        orgO = row[3]

        if person == INperson and org == INorg:
            if orgO in SocietalSector:
                return orgO
    return ''

def getFamilyName(INperson):
    if INperson not in fmDic:
#         print(INperson, " Not found!")
        return ''
    return fmDic[INperson]

resList = []
for row in workList:
    person = row[0]
#     family = row[1]
    org = row[2]
    
    period = getPeriod(person, org)
    if period == '':
        continue
        
    societal = getSocietal(person, org)
    if societal == '':
        continue

    familyName = getFamilyName(person)
    if familyName == '':
        continue
        
    resList.append([person, familyName, org, societal, period])
    
print(len(resList), resList[0:5])

7346 [['林熊祥', '林維讓家系', '林本源製糖會社', 'PrivateBusiness', 'JapaneseColonialOrganization'], ['林熊祥', '林維讓家系', '林本源製糖會社', 'PrivateBusiness', 'JapaneseColonialOrganization'], ['林熊祥', '林維讓家系', '台灣大有物產株式會社', 'PrivateBusiness', 'JapaneseColonialOrganization'], ['林熊祥', '林維讓家系', '台灣大有物產株式會社', 'PrivateBusiness', 'JapaneseColonialOrganization'], ['林熊祥', '林維讓家系', '台北商事株式會社', 'PrivateBusiness', 'JapaneseColonialOrganization']]


In [17]:
orgDf = pd.DataFrame(resList, columns=['Person', 'FamilyName', 'Organization', 'SocietalSector', 'OrganizationPeriod'])
orgDf.drop_duplicates(keep='first', inplace=True)
orgDf.sort_values(by=['FamilyName', 'Person'], inplace=True)
orgDf.head()

,Person,FamilyName,Organization,SocietalSector,OrganizationPeriod
3813,何傳,何傳家系,高雄市,PublicAdministration,BothPeriods
3815,何傳,何傳家系,高雄州,PublicAdministration,JapaneseColonialOrganization
3817,何傳,何傳家系,民報商事社,PrivateBusiness,JapaneseColonialOrganization
3819,何傳,何傳家系,台灣木工株式會社,PrivateBusiness,JapaneseColonialOrganization
3821,何傳,何傳家系,永豐商店理事會,PrivateBusiness,JapaneseColonialOrganization


In [18]:
write_file_to = '{0}/{1}'.format(filepath, 'personFamilyOrgPeriod_20190521_v4.xlsx')
orgDf.to_excel(write_file_to, index=False)